In [1]:
import pygame
import numpy
import random
import sys
from CryptoFunctions import *
COIN = 'BTCUSDT'

DATA = get_candles(getdata(COIN,'1m', 240))

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
SCREENH = 600
SCREENW = 1500
MARGINS = 0
GRID = 1

PURPLE = (160,70,160)
RED = (255,30,30)
GREEN = (130,255,150)
BLUE = (130,150,255)
BLACK = (0,0,10)
GRAY = (35,35,35)
DARKGRAY = (115,115,115)
WHITE = (255,255,255)
ORANGE = (150,70,30)
LIGHTBLUE = (70,210,240)
LIGHTGREEN = (100,240,70)
PINK = (230,40,140)
YELLOW = (240,240,10)
LIGHTRED = (255,100,100)


COLORS = [PURPLE,RED,GREEN,BLUE,YELLOW]
GRAPHHEIGHT = 400
MARGINHEIGHT = 100

POINTWIDTH = 1
POINTHEIGHT = 1


#DATA = extract_features(getdatainterval(COIN,'1m','1 Sep, 2021','20 Jan, 2023'))


SPEEDTICK = 500
pygame.init()
font = pygame.font.Font('freesansbold.ttf', 12)

COMISSION = 0.025
STARTPOS = 600

In [3]:
class Point:
    def __init__(self, graph, x,y, color,data):
        self.graph = graph
        self.x = x
        self.y = y
        self.color = color
        
        self.win = self.graph.win
        self.inTrade = 0
        self.wInTrade = 3
        self.colorInTrade = WHITE
        self.onLine = 2
        
        
        self.w = POINTWIDTH    
        
        self.data = data
    
    def draw(self):
        self.x = self.x - 1
        if (self.onLine > 0):
            pygame.draw.rect(self.win, YELLOW, [self.x, self.y, 2, 2])
            self.onLine -= 1
        elif self.inTrade == 1:
            pygame.draw.rect(self.win, self.colorInTrade, [self.x, self.y, self.wInTrade, self.wInTrade])
        else:
            pygame.draw.rect(self.win, self.color, [self.x, self.y, self.w, self.w])

In [4]:
class AgainstWaves:
    def __init__(self, graph):
        self.win = graph.win
        self.graph = graph
        
        self.color = PINK
        self.color2 = PINK
        
        self.coordinatesOfLines = []

        self.threshold = 0.2
        self.percentage = 1.0025
        
        self.pricebought = 0
        self.pricesold = 0
        
        self.state = 'out'
        self.operation = 'waiting'
        
        self.dataintake = ''
        self.positionX = STARTPOS
        self.position = 0
        self.balance = 1000
        self.indicatorsHeight = 80
        
        self.text = 'Started'
        self.poses = []
        
        self.w = 1

        self.actions = []
        
    def draw(self):
        self.dataintake = self.graph.data.iloc[self.position]
        self.graph.points[self.position].onLine = 10

        if self.state == 'out':
            if self.dataintake.p <= -1*self.threshold:
                self.operation = 'long'
                self.pricebought = self.dataintake.c
                self.text = 'Longed at ' + str(self.pricebought)
                self.state = 'in'
                self.graph.points[self.position].inTrade = True
                self.coordinatesOfLines.append([GREEN,'long',self.graph.points[self.position]])
                self.poses.append(self.position)


                
                
                self.color2 = GREEN

            elif self.dataintake.p >= self.threshold:
                self.operation = 'short'
                self.pricebought = self.dataintake.c
                self.text = 'Shorted at ' + str(self.pricebought)
                self.state = 'in'
                self.graph.points[self.position].inTrade = True
                self.coordinatesOfLines.append([RED,'short',self.graph.points[self.position]])
                self.poses.append(self.position)


                
                
                self.color2 = RED

            else:
                self.operation = 'wait'
                self.color2 = PINK

                self.text = 'Waiting'
                self.state = 'out'
        
        elif self.state == 'in':
            if (self.dataintake.c/self.pricebought >= self.percentage) or (self.pricebought/self.dataintake.c >= self.percentage):
                if self.operation == 'long':
                    x = round(float(self.dataintake.c/self.pricebought),5)
                    self.text = "Closed "+ str((x-1.0)*20 + 1.0 - COMISSION)
                    self.balance = self.balance * (float(self.text.split(' ')[1]))

                    
                    self.state = 'out'
                    self.color2 = PINK
                    self.graph.points[self.position].inTrade = True
                    
                    self.coordinatesOfLines[-1].append(self.graph.points[self.position])
                    
                    self.poses.append(-1*self.position)
                elif self.operation == 'short':
                    x = round(float(self.pricebought/self.dataintake.c),5)
                    self.text = "Closed "+ str((x-1.0)*20 + 1.0 - COMISSION)
                    self.balance = self.balance * (float(self.text.split(' ')[1]))

                    self.state = 'out'
                    self.color2 = PINK
                    self.graph.points[self.position].inTrade = True

                    self.coordinatesOfLines[-1].append(self.graph.points[self.position])
                    
                    self.poses.append(-1*self.position)

        self.actions.append(self.operation)
        self.position += 1
        
        pygame.draw.line(self.win, self.color, (self.positionX,0+MARGINHEIGHT), (self.positionX, self.graph.points[self.position].y),1)
        pygame.draw.rect(self.win, self.color2, [self.positionX - 8, GRAPHHEIGHT + MARGINHEIGHT, 16, 16])

        for i in range(len(self.coordinatesOfLines)):
            x = self.coordinatesOfLines[i]
            if len(x) < 4:
                break
            startpoint = x[2]
            endpoint = x[3]
            if endpoint.x > -100:
                pygame.draw.line(self.win, x[0], (startpoint.x,startpoint.y),(endpoint.x,endpoint.y),1)
                if x[1] == 'long':
                    if endpoint.y < startpoint.y:
                        ind_color = LIGHTGREEN
                    else:
                        ind_color = BLUE

                else:
                    if endpoint.y > startpoint.y:
                        ind_color = LIGHTGREEN
                    else:
                        ind_color = BLUE
                pygame.draw.rect(self.win, ind_color, [startpoint.x, self.indicatorsHeight, 1,10])
            textOperation = font.render(self.text, True, WHITE, BLACK)
            textOperationRect = textOperation.get_rect()
            
            self.win.blit(textOperation, textOperationRect)
            text = font.render(str(int(self.balance)), True, WHITE, BLACK)
            
            textPrice = font.render(str(float(self.graph.points[self.position].data.c)), True, WHITE, BLACK)
            textrectPrice = textPrice.get_rect()
            textrectPrice.center = (self.positionX, SCREENH - MARGINHEIGHT + 45)

            textrect = text.get_rect()
            textrect.center = (self.positionX, SCREENH - MARGINHEIGHT + 30)
            self.win.blit(text, textrect)
            self.win.blit(textPrice, textrectPrice)

        
        

In [5]:
class WithWaves:
    def __init__(self, graph):
        self.win = graph.win
        self.graph = graph
        
        self.color = PURPLE
        self.color2 = PURPLE
        
        self.coordinatesOfLines = []

        self.threshold = 0.2
        self.percentage = 1.0025
        
        self.pricebought = 0
        self.pricesold = 0
        
        self.state = 'out'
        self.operation = 'waiting'
        
        self.dataintake = ''
        self.positionX = STARTPOS + 50
        self.position = 50
        self.balance = 1000
        self.indicatorsHeight = 80
        
        self.text = 'Started'
        self.poses = []
        
        self.w = 1

        self.actions = []
        
    def draw(self):
        self.dataintake = self.graph.data.iloc[self.position]
        self.graph.points[self.position].onLine = 10

        if self.state == 'out':
            if self.dataintake.p >= self.threshold:
                self.operation = 'long'
                self.pricebought = self.dataintake.c
                self.text = 'Longed at ' + str(self.pricebought)
                self.state = 'in'
                self.graph.points[self.position].inTrade = True
                self.coordinatesOfLines.append([GREEN,'long',self.graph.points[self.position]])
                self.poses.append(self.position)


                
                
                self.color2 = GREEN

            elif self.dataintake.p <= -1*self.threshold:
                self.operation = 'short'
                self.pricebought = self.dataintake.c
                self.text = 'Shorted at ' + str(self.pricebought)
                self.state = 'in'
                self.graph.points[self.position].inTrade = True
                self.coordinatesOfLines.append([RED,'short',self.graph.points[self.position]])
                self.poses.append(self.position)


                
                
                self.color2 = RED

            else:
                self.operation = 'wait'
                self.color2 = PURPLE

                self.text = 'Waiting'
                self.state = 'out'
        
        elif self.state == 'in':
            if (self.dataintake.c/self.pricebought >= self.percentage) or (self.pricebought/self.dataintake.c >= self.percentage):
                if self.operation == 'long':
                    x = round(float(self.dataintake.c/self.pricebought),5)
                    self.text = "Closed "+ str((x-1.0)*20 + 1.0 - COMISSION)
                    self.balance = self.balance * (float(self.text.split(' ')[1]))

                    
                    self.state = 'out'
                    self.color2 = PURPLE
                    self.graph.points[self.position].inTrade = True
                    
                    self.coordinatesOfLines[-1].append(self.graph.points[self.position])
                    
                    self.poses.append(-1*self.position)
                elif self.operation == 'short':
                    x = round(float(self.pricebought/self.dataintake.c),5)
                    self.text = "Closed "+ str((x-1.0)*20 + 1.0 - COMISSION)
                    self.balance = self.balance * (float(self.text.split(' ')[1]))

                    self.state = 'out'
                    self.color2 = PURPLE
                    self.graph.points[self.position].inTrade = True

                    self.coordinatesOfLines[-1].append(self.graph.points[self.position])
                    
                    self.poses.append(-1*self.position)

        self.actions.append(self.operation)
        self.position += 1
        
        pygame.draw.line(self.win, self.color, (self.positionX,0+MARGINHEIGHT), (self.positionX, self.graph.points[self.position].y),1)
        pygame.draw.rect(self.win, self.color2, [self.positionX - 8, GRAPHHEIGHT + MARGINHEIGHT, 16, 16])

        for i in range(len(self.coordinatesOfLines)):
            x = self.coordinatesOfLines[i]
            if len(x) < 4:
                break
            startpoint = x[2]
            endpoint = x[3]
            if endpoint.x > -100:
                pygame.draw.line(self.win, x[0], (startpoint.x,startpoint.y),(endpoint.x,endpoint.y),1)
                if x[1] == 'long':
                    if endpoint.y < startpoint.y:
                        ind_color = LIGHTGREEN
                    else:
                        ind_color = BLUE

                else:
                    if endpoint.y > startpoint.y:
                        ind_color = LIGHTGREEN
                    else:
                        ind_color = BLUE
                pygame.draw.rect(self.win, ind_color, [startpoint.x, self.indicatorsHeight, 1,10])
            textOperation = font.render(self.text, True, WHITE, BLACK)
            textOperationRect = textOperation.get_rect()
            
            self.win.blit(textOperation, textOperationRect)

            text = font.render(str(int(self.balance)), True, WHITE, BLACK)
            
            textPrice = font.render(str(float(self.graph.points[self.position].data.c)), True, WHITE, BLACK)
            textrectPrice = textPrice.get_rect()
            textrectPrice.center = (self.positionX, SCREENH - MARGINHEIGHT + 45)

            textrect = text.get_rect()
            textrect.center = (self.positionX, SCREENH - MARGINHEIGHT + 30)
            self.win.blit(text, textrect)
            self.win.blit(textPrice, textrectPrice)

        
        

In [6]:
class Graph:
    def __init__(self, data):
        self.win = pygame.display.set_mode((SCREENW,SCREENH))
        self.win.fill(GRAY)
        pygame.display.set_caption('Graph')
        self.clock = pygame.time.Clock()
        
        self.moved = 0
        self.data = data
        self.over = False
        self.hgraph = GRAPHHEIGHT
        self.ps = [1.0]
        self.ns = [100]
        self.heights = []
        self.points = []
        
        
        self.algos = [WithWaves(self),AgainstWaves(self)]
        
        self.cs = []

        self.reset()
        
    def reset(self):
        for p in range(len(self.data.p.values)):
            self.ps.append(round(self.data.p.values[p]/100 + 1,5) * self.ps[p])
            self.cs.append(self.data.c.values[p])
        maxp = (max(self.ps))
        minp = (min(self.ps))
        maxc = (max(self.cs))
        minc = (min(self.cs))
        self.heights = [-(((c-minc)/(maxc - minc) * (SCREENH - 2 * 
                           MARGINHEIGHT)) - SCREENH + MARGINHEIGHT) for c in self.cs]
        for i in range(len(self.cs)):
            position = i + MARGINS
            height = self.heights[i]
            if i != 0:
                color = GREEN if self.data.p.values[i-1] >= 0.0 else RED
            else:
                color = WHITE
            self.points.append(Point(self,position+STARTPOS,height,color,self.data.iloc[i]))
        self.draw()    
    
    
    def play(self):
        self.draw()
        
    def draw(self):
        self.clock.tick(SPEEDTICK)

        self.win.fill(GRAY)
        for point in self.points:
            if point.x > -100:
                point.draw()
        for algo in self.algos:
            algo.draw()
        pygame.draw.rect(self.win, PURPLE, [0, SCREENH-MARGINHEIGHT, SCREENW,2])
        pygame.draw.rect(self.win, PURPLE, [0, MARGINHEIGHT, SCREENW,2])
        


        self.moved += 1
        


        pygame.display.update()
    
        
        

In [7]:
graph = Graph(DATA)

In [8]:
for i in range(len(DATA)):
#for i in range(500):
    graph.play()


IndexError: list index out of range